<a href="https://colab.research.google.com/github/stiflerGit/CAD-Mammography/blob/master/InceptionResNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')

work_dir = "./drive/My Drive/RESnet/"
if not os.path.exists(work_dir):
  os.makedirs(work_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

"""
This script goes along my blog post:
Keras InceptionResetV2 (https://jkjung-avt.github.io/keras-inceptionresnetv2/)
"""
import os

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

VERSION = '0.0.1'

DATASET_PATH  = '/content/drive/My Drive/CI_FinalProject/'
IMAGE_SIZE    = (224, 224)
NUM_CLASSES   = 2
BATCH_SIZE    = 8  # try reducing batch size or freeze more layers if your GPU runs out of memory
FREEZE_LAYERS = 2  # freeze the first this many layers for training
NUM_EPOCHS    = 20
WEIGHTS_FINAL = 'PT-CNN%s.h5' % VERSION

TRAIN_IMGS_FILE_NAME = 'train_img_%d.npy' % IMAGE_SIZE[0]
TRAIN_LABELS_FILE_NAME = 'train_lab.npy'
TEST_IMGS_FILE_NAME = 'public_test_image_%d.npy' % IMAGE_SIZE[0]
TEST_LABELS_FILE_NAME = 'public_test_label.npy'


"""
    DATA PRE-PROCESSING
    """

import numpy as np
from sklearn.model_selection import train_test_split

train_gray_imgs = np.load(os.path.join(DATASET_PATH, TRAIN_IMGS_FILE_NAME))
train_lbls = np.load(os.path.join(DATASET_PATH, TRAIN_LABELS_FILE_NAME))

# print(train_lbls.shape) # 2864 images (it is a quite small dataset)
# print(np.count_nonzero(train_lbls)) # 1546 
# 1546 images belong to class 1, this means 1318 images belong to class 0
# so our class distribution is good

# we have to equally distribute otherwise the error:
### Training and validation subsets have different number of classes after the 
### split. If your numpy arrays are sorted by the label, you might want to shuffle them.
# could be raised. This is due to the distribution of classes of the labels.
# for example after the split, the training labels set contains both class 1 and 2,
# and on the other hand validation labels set contains either class 1 or class 2.
# Even if the train data would cointain both the classes it's better to equally 
# distribute the two classes over the sets.
val_split = 0.2
X_train, X_val, y_train, y_val = train_test_split(train_gray_imgs, train_lbls,
                                                  test_size=val_split, 
                                                  stratify=train_lbls)

X = np.concatenate((X_train, X_val))
y = np.concatenate((y_train, y_val))

train_samples = len(X_val)
valid_samples = len(X_val)
del(X_train, X_val)
del(y_train, y_val)

# this is done to transform the gray imgs into rgb images
# X = X.astype('float32')/65536
X = np.stack((X,)*3, axis=-1)

del(train_gray_imgs)
del(train_lbls)

test_gray_imgs = np.load(os.path.join(DATASET_PATH, TEST_IMGS_FILE_NAME))
test_rgb_imgs = np.stack((test_gray_imgs,)*3, axis=-1)
test_lbls = np.load(os.path.join(DATASET_PATH, TEST_LABELS_FILE_NAME))

del(test_gray_imgs)

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=40, 
    zoom_range=0.2,
    rescale=1./65536, 
    horizontal_flip=True, vertical_flip=True, 
    validation_split=val_split)

train_batches = train_datagen.flow(X, y, batch_size=BATCH_SIZE, subset='training')
valid_batches = train_datagen.flow(X, y, batch_size=BATCH_SIZE, subset='validation')

test_datagen = ImageDataGenerator(rescale=1./65536)
test_batches = test_datagen.flow(test_rgb_imgs, test_lbls, batch_size=BATCH_SIZE)

test_samples = len(test_rgb_imgs)

# show class indices
# print('****************')
# for cls, idx in train_batches.class_indices.items():
#     print('Class #{} = {}'.format(idx, cls))
# print('****************')

Using TensorFlow backend.


In [6]:

# build our classifier model based on pre-trained InceptionResNetV2:
# 1. we don't include the top (fully connected) layers of InceptionResNetV2
# 2. we add a DropOut layer followed by a Dense (fully connected)
#    layer which generates softmax class score for each class
# 3. we compile the final model using an Adam optimizer, with a
#    low learning rate (since we are 'fine-tuning')
net = InceptionResNetV2(include_top=False,
                        weights='imagenet',
                        input_tensor=None,
                        input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))

for layer in net.layers:
    layer.trainable = False

x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)
net_final = Model(inputs=net.input, outputs=output_layer)
'''
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True
   '''

net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(net_final.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_609 (Conv2D)             (None, 111, 111, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1_609 (Bat (None, 111, 111, 32) 96          conv2d_609[0][0]                 
__________________________________________________________________________________________________
activation_609 (Activation)     (None, 111, 111, 32) 0           batch_normalization_v1_609[0][0] 
__________________________________________________________________________________________________
conv2d_610

In [7]:
# train the model
history = net_final.fit_generator(train_batches,
                        steps_per_epoch = train_samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS)

# save trained weights
# net_final.save(WEIGHTS_FINAL)

Epoch 1/20
287/287 [==============================] - 81s 283ms/step - loss: 0.7746 - acc: 0.6134 - val_loss: 0.8697 - val_acc: 0.6154
Epoch 2/20
287/287 [==============================] - 73s 254ms/step - loss: 0.5911 - acc: 0.7155 - val_loss: 0.7634 - val_acc: 0.6608
Epoch 3/20
287/287 [==============================] - 73s 254ms/step - loss: 0.5234 - acc: 0.7544 - val_loss: 0.5924 - val_acc: 0.6976
Epoch 4/20
287/287 [==============================] - 73s 254ms/step - loss: 0.5310 - acc: 0.7469 - val_loss: 0.6623 - val_acc: 0.6906
Epoch 5/20
287/287 [==============================] - 73s 254ms/step - loss: 0.5061 - acc: 0.7696 - val_loss: 0.6455 - val_acc: 0.6836
Epoch 6/20
287/287 [==============================] - 73s 254ms/step - loss: 0.4800 - acc: 0.7784 - val_loss: 0.6435 - val_acc: 0.6941
Epoch 7/20
287/287 [==============================] - 73s 253ms/step - loss: 0.4644 - acc: 0.7805 - val_loss: 0.5909 - val_acc: 0.7080
Epoch 8/20
287/287 [==============================] - 7

In [8]:
results = net_final.evaluate_generator(
    test_batches, 
    steps=test_samples // BATCH_SIZE, 
    verbose=1)

print("test_loss: ", results[0], " test_acc: ", results[1])

44/44 [==============================] - 8s 193ms/step - loss: 0.5503 - acc: 0.7443
test_loss:  0.5503416047854857  test_acc:  0.7443182
